<b>Importing Librabries</b>

In [16]:
import tensorflow as tf
import cv2 as cv
import os
from keras.layers import Dropout,Conv2D,Flatten,MaxPooling2D,Dense
from keras.models import Sequential,Model
from keras.applications import ResNet50V2
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

<b>Defining Hyper Parameters</b>

In [6]:
params = {
    'TRAIN_DATASET':'data/train_filtered',
    'VAL_DATASET':'data/val_filtered',
    'epochs':10,
    'batch_size':16,
    'target_size':[256,256]
}

<b>Applying Image Augmentation and Preprocessing</b>


In [7]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    channel_shift_range=0.3,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1. / 255,
)

val_datagen =  tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
)

In [9]:
train = train_datagen.flow_from_directory(
    directory = params['TRAIN_DATASET'],
    target_size=params['target_size'],
    class_mode='categorical',
    shuffle=True,
    batch_size=params['batch_size']
)

validation = train_datagen.flow_from_directory(
    directory = params['VAL_DATASET'],
    target_size=params['target_size'],
    class_mode='categorical',
    shuffle=True,
    batch_size=params['batch_size']
)

Found 8178 images belonging to 120 classes.
Found 2044 images belonging to 120 classes.


<b>Setting Up ResNet50V2 Model for Transfer Learning</b>

In [10]:
resnet_model = ResNet50V2(
    input_shape=(params['target_size'][0], params['target_size'][1], 3),
    include_top=False,
    weights="imagenet"
)

94674944/94668760 [==============================] - 278s 3us/step


In [12]:
for layer in resnet_model.layers:
    layer.trainable = False 

<b>Adding Dense Layer at the bottom of ResNet50V2 Model</b>

In [14]:
last_layer = resnet_model.output

last_layer = Flatten()(last_layer)
last_layer = Dense(512,activation='relu')(last_layer)
last_layer = Dropout(0.35)(last_layer)
last_layer = Dense(256,activation=tf.nn.relu)(last_layer)
last_layer = Dropout(0.2)(last_layer)
last_layer = Dense(64,activation='relu')(last_layer)
last_layer = Dense(120,activation='softmax')(last_layer)

model = Model(resnet_model.input, last_layer)#first layer,last layer

<b>Configuring Model with callbacks and other hyper parameter</b>

In [17]:
checkpoint = ModelCheckpoint(
                             'Model/emotion_face_mobilNet.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

In [ ]:
model.compile(optimizer = 'adam', 
              loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train,
    epochs=params['epochs'],
    validation_data=validation,
    callbacks=callbacks
)